## Convertational Chatbot With Memory:

**Objectives:**
- Use different messages type: AIMessage and HumanMessage
- Maintain full converstation history.
- Create sophisticated converstational loop.

**Imports**

In [1]:
from typing import TypedDict, List

from langgraph.graph import StateGraph, START, END


from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage

**State and Node Function**

In [2]:
class AgentState(TypedDict):
    messages: List[HumanMessage]


# use a tool call model: ~/..$ ollama ls
llm = ChatOllama(model='llama3.2:1b')

def process(state: AgentState) -> AgentState:
    response = llm.invoke(state['messages'])
    print(f"\nAI : {response.content}")

    # Appends AI output in messages
    state['messages'].append(AIMessage(content=response.content))
    return state

**Graph, Nodes and Edges**

In [3]:
graph = StateGraph(AgentState)

# nodes
graph.add_node('process', process)

# edges
graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()

**Running our agent**

In [4]:
converstation_history = []

while (user_input := input("User: ")) != "exit":
    print(f"\nUser: {user_input}")

    # Appends new user message in conv history and invokes
    converstation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({
        "messages": converstation_history
    })

    print("result::", result["messages"])

    # note that, state messages contains all the messages including the response
    # check the process node
    converstation_history = result["messages"]


User: Hi, i'm bob

AI : Hello Bob. How's it going? Is there something I can help you with or would you like to chat about something in particular?
result:: [HumanMessage(content="Hi, i'm bob", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Bob. How's it going? Is there something I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={})]

User: Add 435345 34534

AI : Let's add those numbers together.

435345 + 34534 = 469379
result:: [HumanMessage(content="Hi, i'm bob", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Bob. How's it going? Is there something I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Add 435345 34534', additional_kwargs={}, response_metadata={}), AIMessage(content="Let's add those numbers together.\n\n435345 + 34534 = 469379", additional_kwargs={}, response_meta